In [1]:
using LinearAlgebra

In [2]:
function int_to_binarray(x::Int,y::Int)
    out = Array{Int8}(undef, y)
    t = x
    for i in 1:y
        out[i] = t % 2
        t ÷= 2
    end
    return out
end

function binarray_to_int(x)
    out = sum(x .* (2 .^ Vector(0:length(x)-1)))
end

binarray_to_int (generic function with 1 method)

In [3]:
struct AdditionProblem
    x1::Vector{Int8}
    x2::Vector{Int8}
    y::Vector{Int8}
end
function AdditionProblem(x1::Number, x2::Number, siz::Number)
    x1_in = Int64(x1)
    x2_in = Int64(x2)
    siz_in = Int64(siz)
    y_in = x1_in + x2_in
    return AdditionProblem(int_to_binarray(x1_in, siz_in), int_to_binarray(x2_in, siz_in), int_to_binarray(y_in, siz_in))
end

function AdditionProblem(x1::Number, x2::Number)
    return Problem(x1, x2, 8)
end

struct IncrementProblem
    x::Vector{Int8}
    y::Vector{Int8}
end
function IncrementProblem(x::Number; size::Number)
    x_in = Int64(x)
    siz_in = Int64(size)
    y_in = x_in + 1
    return IncrementProblem(int_to_binarray(x_in, siz_in), int_to_binarray(y_in, siz_in))
end

function IncrementProblem(x::Number)
    return IncrementProblem(x, 8)
end

function IncrementProblem(x::IncrementProblem)
    return IncrementProblem(binarray_to_int(x.y), size=length(x.x))
end

function isSolution(p::IncrementProblem, vec)
    n = length(p.x)
    if length(vec)<(2*n + 1)
        return false
    end
    return (vec[1] == 1) && (vec[2:n+1]==p.x) && (vec[2+n:2*n+1]==p.y)
end

isSolution (generic function with 1 method)

In [4]:
function randomProblem(n)
    return IncrementProblem(floor(rand()*2^n), size=n)
end

function randomProblemMatrix(p)
    n = length(p.x)
    mat_size = 1+4*n
    out_mat = zeros(Float16, (mat_size, mat_size))
    out_mat[2+n:end, 1:end] = floor.(rand(mat_size-1-n, mat_size).*3 .-1)
    out_mat[1,1] = 1
    for k = 1:n
        out_mat[k+1,k+1] = 1
        out_mat[1+n+k,k+n+1] = 0
    end
    return out_mat
end

function randomProblemVector(p)
    n = length(p.x)
    vec_size = 1+2*n+3*(n-1)
    out_vec = zeros(Float16, vec_size)
    out_vec[1] = 1
    for k = 1:n
        out_vec[k+1] = p.x[k]
        out_vec[k+1+n] = p.y[k]
    end
    for k = 2+2n:vec_size
        out_vec[k] = min(max(rand()*3-1, 0), 1)
    end
    return out_vec
end

randomProblemVector (generic function with 1 method)

Now we need to start constructing a neural network. The simple neural network will be a vector of nodes with a dense matrix of weights. This will probably turn into a sparse matrix for large numbers of nodes, but with a small number of nodes it's faster to do it as a dense matrix.

In [5]:
function increment_network(vec, mat, func, n_static)
    temp = mat * vec
    for i in 1:length(vec)
        if i <= n_static
            temp[i] = vec[i]
        else
            temp[i] = func(temp[i])
        end
    end
    return temp
end
σ = (x)->atan(x)*4/π
ξ = (x)-> max(min(1.0,x),0.0)
function increment_network(vec, mat, n_static)
    return increment_network(vec, mat, ξ, n_static)
end

increment_network (generic function with 2 methods)

In [6]:
function isStableSolution(p, v, m)
    n = length(p.x)
    if !isSolution(p, v)
        return false
    end
    v_out = increment_network(v,m,n)
    if v != v_out
        return false
    elseif !isSolution(p,v_out)
        return false
    end
    return true
end

isStableSolution (generic function with 1 method)

I am going to start with the incremental problem, searching for the pattern of adding 1 to an array.
This one is a simple chain of XOR and AND.

NOT(a) = 1 - a
OR(a,b) = a + b
AND(a,b) = a+b-1
NAND(a,b) = 2 - a - b
XOR(a,b) = AND( OR(a,b), NOT( AND(a,b) ) )

First 9 elements are static (1 "rail" and 8 input)
Elements 10:17 are output

Lets start with 4 elements:

In [7]:
function construct_solution_matrix(n)
    mat_size = 1+2*n+3*(n-1)
    out_mat = zeros(Float16, (mat_size, mat_size))
    for k = 1:n+1
        out_mat[k,k] = 1
    end
    out_mat[2+n, 1] = 1
    out_mat[2+n,2] = -1
    old_and = 2
    for k = 2:n
        and_row = -4 + 2*n + 3*k
        or_row = and_row+1
        not_and_row = or_row+1
        output_row = 1+n+k
        out_mat[and_row, 1] = -1
        out_mat[and_row, 1+k] = 1
        out_mat[and_row, old_and] = 1
        out_mat[or_row, 1+k] = 1
        out_mat[or_row, old_and] = 1
        out_mat[not_and_row, 1] = 1
        out_mat[not_and_row, and_row] = -1
        old_and = and_row
        out_mat[output_row, 1] = -1
        out_mat[output_row, or_row] = 1
        out_mat[output_row, not_and_row] = 1
    end
    return out_mat
end

construct_solution_matrix (generic function with 1 method)

In [19]:
n = 128
p = randomProblem(n)
v = randomProblemVector(p)
m = construct_solution_matrix(n)
for q = 1:50
    v = increment_network(v,m,n)
end
print(isStableSolution(p,v,m))

true

Next problems:

1. How do we intelligently increase the size of the network to handle more solutions using previously known solutions to restrict the problem space
2. Optimize the network to eliminate redundancy without sacrificing accuracy